# Modules à importer

In [1]:
import math
import scipy
import pickle
import astropy
import numpy as np
import pandas as pd
import iminuit as im
import ipywidgets as ipw
from importlib import reload
from scipy.stats import poisson
from variaIa import baserate
from variaIa import stretchevol
from astropy.cosmology import Planck15
from IPython.display import display, Latex
import matplotlib.pyplot as plt
import random

## Définitions pandas, surveys, $z[:z_{\text{max}}]$, $x_1[:z_{\text{max}}]$, mean, std et données totales

In [2]:
cons = ipw.Checkbox(
    value=False,
    description='Conservative')

In [3]:
d = pd.read_csv('../../Data/data_cheat.csv', sep=' ', index_col = 'CID')
d_snf = pd.read_csv('../../Data/lssfr_paper_full_sntable.csv',sep=',')

surveys = ['SNF', 'SDSS', 'PS1', 'SNLS', 'HST']

surv = {'SNF':  d_snf.loc[d_snf['name'].str.contains('SNF|LSQ|PTF',na=False,regex=True)],
        'SDSS': d[d['IDSURVEY'] == 1],
        'PS1':  d[d['IDSURVEY'] == 15],
        'SNLS': d[d['IDSURVEY'] == 4],
        'HST':  d[d['IDSURVEY'].isin([101, 100, 106])]}

dgmap = plt.cm.get_cmap('viridis')
colors = {'SNF': dgmap(0),
          'SDSS': dgmap(50),
          'PS1': dgmap(125),
          'SNLS': dgmap(200),
          'HST': dgmap(300)}

with open('../../Data/zmax_mlim', 'rb') as f:
    z_max = pickle.load(f)
z_max['HST'] = [10, 10]

def set_cons(cons):
    global df
    
    zmax_cuts = dict()
    z_zcuts = dict()
    x1_zcuts = dict()
    x1_err_zcuts = dict()
    
    names = ['SNF' for i in range(len(surv['SNF']['host.zcmb']))]
    stretchs = list(surv['SNF']['salt2.X1'])
    stretchs_err = list(surv['SNF']['salt2.X1.err'])
    redshifts = list(surv['SNF']['host.zcmb'])
    infor = list(surv['SNF']['p(prompt)'])
    py = list(surv['SNF']['p(prompt)'])
    lssfr = list(surv['SNF']['lssfr'])
    lssfr_err_d = list(surv['SNF']['lssfr.err_down'])
    lssfr_err_u = list(surv['SNF']['lssfr.err_up'])
    
    if cons:
        for survey in surveys[1:]:
            zmax_cuts[survey] = np.where(surv[survey].zCMB.values < z_max[survey][0])
            z_zcuts[survey] = surv[survey].zCMB.values[zmax_cuts[survey]]
            x1_zcuts[survey] = surv[survey].x1.values[zmax_cuts[survey]]
            x1_err_zcuts[survey] = surv[survey].x1ERR.values[zmax_cuts[survey]]
    else:
        for survey in surveys[1:]:
            zmax_cuts[survey] = np.where(surv[survey].zCMB.values < z_max[survey][-1])
            z_zcuts[survey] = surv[survey].zCMB.values[zmax_cuts[survey]]
            x1_zcuts[survey] = surv[survey].x1.values[zmax_cuts[survey]]
            x1_err_zcuts[survey] = surv[survey].x1ERR.values[zmax_cuts[survey]]
    
    for survey in surveys[1:]:
        names += [survey for i in range(len(z_zcuts[survey]))]
        stretchs += list(x1_zcuts[survey])
        stretchs_err += list(x1_err_zcuts[survey])
        redshifts += list(z_zcuts[survey])
        infor += list(stretchevol.Evol2G2M2S().delta(z_zcuts[survey]))
        py += list([0 for i in range(len(z_zcuts[survey]))])
        lssfr += list([0 for i in range(len(z_zcuts[survey]))])
        lssfr_err_d += list([0 for i in range(len(z_zcuts[survey]))])
        lssfr_err_u += list([0 for i in range(len(z_zcuts[survey]))])
        
    df = pd.DataFrame({'survey': names,
                       'stretchs': stretchs,
                       'stretchs_err': stretchs_err,
                       'redshifts': redshifts,
                       'infor': infor,
                       'py': py,
                       'lssfr': lssfr,
                       'lssfr_err_d': lssfr_err_d,
                       'lssfr_err_u': lssfr_err_u})

    z_mean = np.mean(df.redshifts)
    z_std = np.std(df.redshifts)
    x1_mean = np.mean(df.stretchs)
    x1_std = np.std(df.stretchs)
    
    return(df)
            
int_set_cons = ipw.interactive(set_cons, cons=cons)
display(int_set_cons)

names = ['SNF' for i in range(len(surv['SNF']['host.zcmb'].values))]
stretchs = list(surv['SNF']['salt2.X1'].values)
stretchs_err = list(surv['SNF']['salt2.X1.err'].values)
redshifts = list(surv['SNF']['host.zcmb'].values)
infor = list(surv['SNF']['p(prompt)'])
py = list(surv['SNF']['p(prompt)'])
lssfr = list(surv['SNF']['lssfr'])
lssfr_err_d = list(surv['SNF']['lssfr.err_down'])
lssfr_err_u = list(surv['SNF']['lssfr.err_up'])

for survey in surveys[1:]:
    names += [survey for i in range(len(surv[survey].zCMB.values))]
    stretchs += list(surv[survey].x1.values)
    stretchs_err += list(surv[survey].x1ERR.values)
    redshifts += list(surv[survey].zCMB.values)
    infor += list(stretchevol.Evol2G2M2S().delta(surv[survey].zCMB.values))
    py += list([0 for i in range(len(surv[survey].zCMB.values))])
    lssfr += list([0 for i in range(len(surv[survey].zCMB.values))])
    lssfr_err_d += list([0 for i in range(len(surv[survey].zCMB.values))])
    lssfr_err_u += list([0 for i in range(len(surv[survey].zCMB.values))])

df_full = pd.DataFrame({'survey': names,
                        'stretchs': stretchs,
                        'stretchs_err': stretchs_err,
                        'redshifts': redshifts,
                        'infor': infor,
                        'py': py,
                        'lssfr': lssfr,
                        'lssfr_err_d': lssfr_err_d,
                        'lssfr_err_u': lssfr_err_u})

interactive(children=(Checkbox(value=False, description='Conservative'), Output()), _dom_classes=('widget-inte…

## Get proba

In [4]:
gen = stretchevol.generic()
gen.set_model('Evol3G2M1S')
gen.set_data(df)
base1STOT = gen.fit()

def get_proba(model):
    return np.exp((base1STOT.get_aic() - model.get_aic())/2)

## Modèles assymSURVEY

In [5]:
gen.set_model('Evol1G1M2S')

gen.set_data(df[df['survey'] == 'SNF'])
assymSNF = gen.fit()

gen.set_data(df[df['survey'] == 'PS1'])
assymPS1 = gen.fit()

gen.set_data(df[df['survey'] == 'SDSS'])
assymSDSS = gen.fit()

gen.set_data(df[df['survey'] == 'SNLS'])
assymSNLS = gen.fit()

gen.set_data(df[df['survey'] == 'HST'])
assymHST = gen.fit()

gen.set_data(df)
assymTOT = gen.fit()

gen.set_data(df[df['survey'].isin(['SDSS','PS1','SNLS','HST'])])
assymTOT_nsnf = gen.fit()

## Modèles BaseSURVEY

In [8]:
gen.set_model('Evol3G2M2S')

gen.set_data(assymSNF.pd)
baseSNF = gen.fit()

gen.set_data(assymSDSS.pd)
baseSDSS = gen.fit()

gen.set_data(assymPS1.pd)
basePS1 = gen.fit()

gen.set_data(assymSNLS.pd)
baseSNLS = gen.fit()

gen.set_data(assymHST.pd)
baseHST = gen.fit()

gen.set_data(assymTOT.pd)
baseTOT = gen.fit()

gen.set_data(assymTOT_nsnf.pd)
baseTOT_nsnf = gen.fit()

In [9]:
modèles = [assymSNF, assymSDSS, assymPS1, assymSNLS, assymHST]

assym_comp = pd.DataFrame({'Assym': ['SNf', 'SDSS', 'PS1', 'SNLS', 'HST', 'Total'],
                           'Free param': [len(k.FREEPARAMETERS) for k in modèles] + [np.sum([len(k.FREEPARAMETERS) for k in modèles])],
                           '$\sigma^-$': [round(k.migrad_out[1][1][2],2) for k in modèles] + ['--'],
                           '$\sigma^-_{\mathrm{err}}$': [round(k.migrad_out[1][1][3],2) for k in modèles] + ['--'],
                           '$\sigma^+$': [round(k.migrad_out[1][2][2],2) for k in modèles] + ['--'],
                           '$\sigma^+_{\mathrm{err}}$': [round(k.migrad_out[1][2][3],2) for k in modèles] + ['--'],
                           '$\mu^0$': [round(k.migrad_out[1][0][2],2) for k in modèles] + ['--'],
                           '$\mu^0_{\mathrm{err}}$': [round(k.migrad_out[1][0][3],2) for k in modèles] + ['--'],
                           '$\mathcal{L}$': [round(k.get_logl(),1) for k in modèles] + [round(np.sum([k.get_logl() for k in modèles]),1)],
                           'AIC': [round(k.get_aic(),1) for k in modèles] + [round(np.sum([k.get_aic() for k in modèles]),1)]})

path = '../../Data/assym_comp'
if cons.value:
    path += '_cons'
path += '.dat'
assym_comp.to_csv(path)

assym_comp

,Assym,Free param,$\sigma^-$,$\sigma^-_{\mathrm{err}}$,$\sigma^+$,$\sigma^+_{\mathrm{err}}$,$\mu^0$,$\mu^0_{\mathrm{err}}$,$\mathcal{L}$,AIC
0,SNf,3,1.34,0.13,-0.41,0.1,0.68,0.15,299.3,305.3
1,SDSS,3,1.31,0.11,0.42,0.09,0.72,0.13,445.0,451.0
2,PS1,3,1.01,0.11,-0.52,0.12,0.38,0.16,397.4,403.4
3,SNLS,3,1.41,0.13,0.15,0.13,1.22,0.15,252.7,258.7
4,HST,3,0.76,0.36,0.79,0.35,0.11,0.44,73.7,79.7
5,Total,15,--,--,--,--,--,--,1468.2,1498.2


In [47]:
print('Delta AIC w/ BBC-modeling =',
      round(baseTOT.get_aic()-np.sum([k.get_aic() for k in modèles]),1))

Delta AIC w/ BBC-modeling = -24.1


In [48]:
print('Proba w/ BBC-modeling =',
      np.exp((baseTOT.get_aic() - np.sum([k.get_aic() for k in modèles]))/2))

Proba w/ BBC-modeling = 5.724059891232207e-06


## Another table comp

In [56]:
modelsa = [assymSDSS, assymPS1, assymSNLS, assymHST, assymSNF]
modelsb = [baseSDSS, basePS1, baseSNLS, baseHST, baseSNF]
param_sco = [[1.142, 1.652, 0.104], [0.384, 0.987, 0.505], [0.974, 1.236, 0.283]]

with open('../Data/SNF_results', 'rb') as f:
    res_SNF = pickle.load(f)
    
with open('../Data/ASS_results', 'rb') as f:
    res_ALL= pickle.load(f)

comp_L = pd.DataFrame({'#Data': [len(k.pd) for k in modelsa] +\
                       [np.sum([len(k.pd) for k in modelsa])] +\
                       [np.sum([len(k.pd) for k in modelsa[:-1]])],
                       'Survey': ['SDSS', 'PS1', 'SNLS', 'HST', 'SNf', 'Total', 'Total-SNf'],
                       '$\mathcal{L}$ asym (this)': [round(k.get_logl(),1) for k in modelsa] +\
                       [round(np.sum([k.get_logl() for k in modelsa]),1)] +\
                       [round(np.sum([k.get_logl() for k in modelsa[:-1]]),1)],
                       'Free param ': [len(k.FREEPARAMETERS) for k in modelsa] +\
                       [np.sum([len(k.FREEPARAMETERS) for k in modelsa])] +\
                       [np.sum([len(k.FREEPARAMETERS) for k in modelsa[:-1]])],
                       '$\mathcal{L}$ Base SNf': [round(k.loglikelihood(res_SNF['aa'], res_SNF['mu_1'], res_SNF['sigma_1'], res_SNF['mu_2'], res_SNF['sigma_2']),1) for k in modelsb] +\
                       [round(np.sum([k.loglikelihood(res_SNF['aa'], res_SNF['mu_1'], res_SNF['sigma_1'], res_SNF['mu_2'], res_SNF['sigma_2']) for k in modelsb]),1)] +\
                       [round(np.sum([k.loglikelihood(res_SNF['aa'], res_SNF['mu_1'], res_SNF['sigma_1'], res_SNF['mu_2'], res_SNF['sigma_2']) for k in modelsb[:-1]]),1)],
                       'Free param': [0, 0, 0, 0, 5,
                                      5,
                                      0],
                       '$\mathcal{L}$ asym Sco': [round(k.loglikelihood(p[0], p[1], p[2]),1) for k,p in zip(modelsa, param_sco)] + ['--'] + ['--']+\
                       ['--'] +\
                       [round(np.sum([k.loglikelihood(p[0], p[1], p[2]) for k,p in zip(modelsa, param_sco)]),1)],
                      })

path = '../Data/asym_comp_L'
if cons.value:
    path += '_cons'
path += '.dat'
comp_L.to_csv(path)

comp_L

,#Data,Survey,$\mathcal{L}$ asym (this),Free param,$\mathcal{L}$ Base SNf,Free param,$\mathcal{L}$ asym Sco
0,167,SDSS,445.0,3,452.9,0,457.9
1,160,PS1,397.4,3,405.2,0,397.6
2,102,SNLS,252.7,3,269.5,0,254.5
3,26,HST,73.7,3,75.9,0,--
4,114,SNf,299.3,3,268.6,5,--
5,569,Total,1468.2,15,1472.0,5,--
6,455,Total-SNf,1168.9,12,1203.4,0,1110


In [15]:
modelsa = [assymSDSS, assymPS1, assymSNLS, assymHST, assymSNF]
modelsb = [baseSDSS, basePS1, baseSNLS, baseHST, baseSNF]
param_sco = [[1.142, 1.652, 0.104], [0.384, 0.987, 0.505], [0.974, 1.236, 0.283]]
    
with open('../../Data/ALL_results', 'rb') as f:
    res_ALL = pickle.load(f)

comp_L = pd.DataFrame({'#Data': [len(k.pd) for k in modelsa] +\
                       [np.sum([len(k.pd) for k in modelsa])] +\
                       [np.sum([len(k.pd) for k in modelsa[:-1]])],
                       'Survey': ['SDSS', 'PS1', 'SNLS', 'HST', 'SNf', 'Total', 'Total-SNf'],
                       '$\mathcal{L}$ asym (this)': [round(k.get_logl(),1) for k in modelsa] +\
                       [round(np.sum([k.get_logl() for k in modelsa]),1)] +\
                       [round(np.sum([k.get_logl() for k in modelsa[:-1]]),1)],
                       'Free param ': [len(k.FREEPARAMETERS) for k in modelsa] +\
                       [np.sum([len(k.FREEPARAMETERS) for k in modelsa])] +\
                       [np.sum([len(k.FREEPARAMETERS) for k in modelsa[:-1]])],
                       '$\mathcal{L}$ Base All': [round(k.loglikelihood(res_ALL['aa'], res_ALL['mu_1'], res_ALL['sigma_1'], res_ALL['mu_2'], res_ALL['sigma_2']),1) for k in modelsb] +\
                       [round(np.sum([k.loglikelihood(res_ALL['aa'], res_ALL['mu_1'], res_ALL['sigma_1'], res_ALL['mu_2'], res_ALL['sigma_2']) for k in modelsb]),1)] +\
                       [round(np.sum([k.loglikelihood(res_ALL['aa'], res_ALL['mu_1'], res_ALL['sigma_1'], res_ALL['mu_2'], res_ALL['sigma_2']) for k in modelsb[:-1]]),1)],
                       'Free param': [0, 0, 0, 0, 5,
                                      5,
                                      0],
                       '$\mathcal{L}$ asym Sco': [round(k.loglikelihood(p[0], p[1], p[2]),1) for k,p in zip(modelsa, param_sco)] + ['--'] + ['--']+\
                       ['--'] +\
                       [round(np.sum([k.loglikelihood(p[0], p[1], p[2]) for k,p in zip(modelsa, param_sco)]),1)],
                      })

path = '../../Data/asym_comp_L_all'
if cons.value:
    path += '_cons'
path += '.dat'
comp_L.to_csv(path)

comp_L

,#Data,Survey,$\mathcal{L}$ asym (this),Free param,$\mathcal{L}$ Base All,Free param,$\mathcal{L}$ asym Sco
0,167,SDSS,445.0,3,446.6,0,457.9
1,160,PS1,397.4,3,399.4,0,397.6
2,102,SNLS,252.7,3,262.9,0,254.5
3,26,HST,73.7,3,75.0,0,--
4,114,SNf,299.3,3,272.9,5,--
5,569,Total,1468.2,15,1456.7,5,--
6,455,Total-SNf,1168.9,12,1183.8,0,1110


In [57]:
comp_A = pd.DataFrame({'#Data': [len(k.pd) for k in modelsa] +\
                       [np.sum([len(k.pd) for k in modelsa])] +\
                       [np.sum([len(k.pd) for k in modelsa[:-1]])] +\
                       [np.sum([len(k.pd) for k in modelsa[:-2]])],
                       'Survey': ['SDSS', 'PS1', 'SNLS', 'HST', 'SNf',
                                  'Total',
                                  'Total-SNf',
                                  'Total-SNf-HST'],
                       'AICc asym (this)': [round(k.get_aic(),1) for k in modelsa] +\
                       [round(np.sum([k.get_aic() for k in modelsa]),1)] +\
                       [round(np.sum([k.get_aic() for k in modelsa[:-1]]),1)] +\
                       [round(np.sum([k.get_aic() for k in modelsa[:-2]]),1)],
                       'Free param ': [len(k.FREEPARAMETERS) for k in modelsa] +\
                       [np.sum([len(k.FREEPARAMETERS) for k in modelsa])] +\
                       [np.sum([len(k.FREEPARAMETERS) for k in modelsa[:-1]])] +\
                       [np.sum([len(k.FREEPARAMETERS) for k in modelsa[:-2]])],
                       'AIC Base SNf': [round(k.loglikelihood(res_SNF['aa'], res_SNF['mu_1'], res_SNF['sigma_1'], res_SNF['mu_2'], res_SNF['sigma_2']),1) for k in modelsb[:-1]] + [round(base.get_aic(),1)] +\
                       [round(2*5 + np.sum([k.loglikelihood(res_SNF['aa'], res_SNF['mu_1'], res_SNF['sigma_1'], res_SNF['mu_2'], res_SNF['sigma_2']) for k in modelsb]),1)] +\
                       [round(np.sum([k.loglikelihood(res_SNF['aa'], res_SNF['mu_1'], res_SNF['sigma_1'], res_SNF['mu_2'], res_SNF['sigma_2']) for k in modelsb[:-1]]),1)] +\
                       [round(np.sum([k.loglikelihood(res_SNF['aa'], res_SNF['mu_1'], res_SNF['sigma_1'], res_SNF['mu_2'], res_SNF['sigma_2']) for k in modelsb[:-2]]),1)],
                       'Free param': [0, 0, 0, 0, 5,
                                      5,
                                      0,
                                      0],
                       'AIC asym Sco': [round(2*3 + k.loglikelihood(p[0], p[1], p[2]),1) for k,p in zip(modelsa, param_sco)] + [80.8] + [305.5] +\
                       [round(2*15 + np.sum([k.loglikelihood(p[0], p[1], p[2]) for k,p in zip(modelsa, param_sco)]),1) + 73.7 + 299.3] +\
                       [round(2*12 + np.sum([k.loglikelihood(p[0], p[1], p[2]) for k,p in zip(modelsa, param_sco)]),1) + 73.7] +\
                       [round(2*9 + np.sum([k.loglikelihood(p[0], p[1], p[2]) for k,p in zip(modelsa, param_sco)]),1)],
                       ' Free param ': [len(k.FREEPARAMETERS) for k in modelsa] +\
                       [np.sum([len(k.FREEPARAMETERS) for k in modelsa])] +\
                       [np.sum([len(k.FREEPARAMETERS) for k in modelsa[:-1]])] +\
                       [np.sum([len(k.FREEPARAMETERS) for k in modelsa[:-2]])],
                      })

path = '../Data/asym_comp_A'
if cons.value:
    path += '_cons'
path += '.dat'
comp_A.to_csv(path)

comp_A

,#Data,Survey,AICc asym (this),Free param,AIC Base SNf,Free param,AIC asym Sco,Free param
0,167,SDSS,451.0,3,452.9,0,463.9,3
1,160,PS1,403.4,3,405.2,0,403.6,3
2,102,SNLS,258.7,3,269.5,0,260.5,3
3,26,HST,79.7,3,75.9,0,80.8,3
4,114,SNf,305.3,3,278.6,5,305.5,3
5,569,Total,1498.2,15,1482.0,5,1513.0,15
6,455,Total-SNf,1192.9,12,1203.4,0,1207.7,12
7,429,Total-SNf-HST,1113.2,9,1127.5,0,1128.0,9


## Minimisation du modèle SNF et calcul $\mathcal{L}$ pour surveys

In [17]:
gen.set_model('Evol3G2M2S')
gen.set_data(df[df['survey'] == 'SDSS'])
baseSDSS = gen.fit()

gen.set_data(df[df['survey'] == 'PS1'])
basePS1 = gen.fit()

gen.set_data(df[df['survey'] == 'SNLS'])
baseSNLS = gen.fit()

In [18]:
models = [basePS1, baseSDSS, baseSNLS]

snf_comp = pd.DataFrame({'Base': ['SDSS', 'PS1', 'SNLS', 'Total'],
                         'Free param': [0 for k in models] + [0],
                         '$\mathcal{L}$': [round(k.get_logl(),1) for k in models] + [round(np.sum([k.get_logl() for k in models]),1)],
                         'AICc': [round(k.get_aic(),1) for k in models] + [round(np.sum([k.get_aic() for k in models]),1)]})

path = '../Data/snf_comp'
if cons.value:
    path += '_cons'
path += '.dat'
snf_comp.to_csv(path)

snf_comp

,Base,Free param,ln $\mathcal{L}$,AICc
0,SDSS,0,395.2,405.6
1,PS1,0,444.7,455.0
2,SNLS,0,254.1,264.7
3,Total,0,1094.0,1125.4
